In [1]:
import aiohttp

# Add parent dir to path
import os
import sys
import io

sys.path.insert(0,os.environ['PWD'])

import netaddr

from rpki_analysis.delegated_stats import read_delegated_extended_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        data = read_delegated_extended_stats(io.StringIO(await resp.text()))
        

ClientPayloadError: Response payload is not completed

In [ ]:
data.dtypes

In [ ]:
resources_by_uuid = data.loc[data.afi != 'asn'].groupby(['opaque_id']) \
  .agg({'resource': lambda grp: netaddr.IPSet(grp)})

In [ ]:
resources_by_uuid.resource.apply(lambda x: list(map(str, x.iter_cidrs())))

In [ ]:
resources_by_uuid['ipv4_count'] = resources_by_uuid.resource.apply(lambda res: sum([cidr.size if cidr.version == 4 else 0 for cidr in res.iter_cidrs()]))

In [ ]:
resources_by_uuid['ipv4_count'].reset_index().sort_values(['ipv4_count'])